In [370]:
import numpy as np
from algorithms import steepest_descent, conjugate_gradient, secant, Finite_Difference, armijo
from cost_functions import V_a, gradV_a, V_b, gradV_b
from numpy.linalg import norm, eig
from functools import partial
import time




In [371]:
def augmented_lagrangian(x0, 
                         cost_function, 
                         equality_constraints = None, 
                         inequality_constraints = None,
                         threshold = 1e-3, 
                         sigma_max = 1e12, 
                         track_history = False, 
                         log = False):
    class P:
        def __init__(self, lambd, sigma):
            self.lambd = lambd
            self.sigma = sigma

        def phi(self, x):
            cost = cost_function(x)
            lambda_eq = lambd[:num_ec , :]
            lambda_ineq = lambd[num_ec:num_c , :]
            sigma_eq = sigma[:num_ec , :]
            sigma_ineq = sigma[num_ec:num_c , :]

            ecs = np.array([ec(x) for ec in equality_constraints])
            cost = cost - sum(lambda_eq * ecs) + 0.5 * sum(sigma_eq * ecs**2)
            
            for i, ineq in enumerate(inequality_constraints):
                ic = ineq(x)
                if ic <= lambda_ineq[i] / sigma_ineq[i]:
                    p_i = np.array([-lambda_ineq[i] * ic + 0.5 * sigma_ineq[i] * ic**2])
                else: 
                    p_i = np.array([-0.5 * lambda_ineq[i]**2 / sigma_ineq[i] ])
                cost = cost + p_i

            return cost

    x_history, V_history = [],[]
    num_ec = len(equality_constraints)
    num_ic = len(inequality_constraints)
    num_c = num_ec + num_ic

    lambd = np.zeros((num_c,1))
    sigma = np.ones((num_c,1))

    c = 1e12 * sigma
    x = x0
    minimum = cost_function(x)
    x_history.append(x), V_history.append(minimum)
    j = 0
    while norm(c) > threshold and all(sigma < sigma_max): 
        p = P(lambd, sigma)
        x,_ = steepest_descent(x,
                               p.phi,
                               None,
                               step_size = 'armijo',
                               threshold = 1e-6, 
                               max_iter = 1e3, 
                               fd_method = 'forward')

        previous_cost = c
        inequality_cost = [const(x) for const in inequality_constraints]
        equality_cost = [const(x) for const in equality_constraints]
        c = equality_cost + inequality_cost 

        if norm(c, np.inf) > 0.25 * norm(previous_cost, np.inf):
            for i in range(num_c):
                if np.abs(c[i]) > 0.25 * norm(previous_cost, np.inf):
                    sigma[i] *= 10
            continue
        lambd = lambd - sigma * c

        minimum = cost_function(x)
        x_history.append(x), V_history.append(minimum)
        j += 1
        # if log:
        #     print(f'x = {x}, V(x) = {minimum:.5f}')
            
    if track_history:
        return x, minimum, x_history, V_history
    else:
        return x, cost_function(x)
    

In [372]:
def V_1(x):
    return np.abs(x[0]-1) + np.abs(x[1]-2)

def h1_1(x):
    return x[0]-x[1]**2

def h2_1(x):
    return x[0]**2 + x[1]**2 - 1

x0 = np.array([[1.5], [-1.5]])


cost_function = V_1
inequality_constraints = [h1_1]
equality_constraints = [h2_1]
gradient_function = None

x, minimum = augmented_lagrangian(x0,
                    cost_function, 
                    equality_constraints, 
                    inequality_constraints,
                    log = True)
x, minimum

(array([[0.71594633],
        [0.71594615]]),
 array([1.56810752]))

In [379]:
def V_3(x):
    return np.log(x[0]) - x[1]

def h1_3(x):
    return x[0]-1

def h2_3(x):
    return x[0]**2 + x[1]**2 - 4


x0 = np.array([[2.], [1.5]])


cost_function = V_3
inequality_constraints = [h1_3]
equality_constraints = [h2_3]
gradient_function = None

x, minimum = augmented_lagrangian(x0,
                    cost_function, 
                    equality_constraints, 
                    inequality_constraints,
                    log = True)
x, minimum

/var/folders/k5/vn3k1nw15mv2z0p8ww8lv16r0000gn/T/ipykernel_92684/2501215019.py:2: RuntimeWarning: invalid value encountered in log
  return np.log(x[0]) - x[1]


(array([[nan],
        [nan]]),
 array([nan]))

In [376]:

def v(x):
   return  -x[0]*x[1]

def h1(x):
    return -x[0]-x[1]**2 + 1

def h2(x):
    return x[0]**2 + x[1]**2

x0 = np.array([[1.0], [1.0]])


cost_function = v
inequality_constraints = [h1, h2]
equality_constraints = []
gradient_function = None

x, minimum = augmented_lagrangian(x0,
                    cost_function, 
                    equality_constraints, 
                    inequality_constraints)
x, minimum 



(array([[0.6725883 ],
        [0.57990867]]),
 array([-0.39003979]))